In [1]:
#Importing Packages and Checking GPU
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np
from scipy import io
from tensorflow.keras.callbacks import EarlyStopping

GPUDevices = tf.config.list_physical_devices('GPU')

if GPUDevices:
	print("TensorFlow is using the GPU")
	for i,gpu in enumerate(GPUDevices):
		print(f"GPU {i} Found: {gpu.name}")
else:
	print("No GPU Devices found. TensorFlow is using the CPU")

2025-08-29 23:38:12.492616: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-29 23:38:12.501277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756490892.511542  406132 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756490892.514669  406132 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756490892.522517  406132 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

No GPU Devices found. TensorFlow is using the CPU


W0000 00:00:1756490893.512049  406132 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
#Loading DataSet
print("Loading EMNIST DataSet")
dataset = io.loadmat('datasets/emnist-letters.mat')
struct_dataset = dataset['dataset'][0,0]

TrainImages, TrainLabels = struct_dataset['train'][0,0]['images'], struct_dataset['train'][0,0]['labels']
TestImages, TestLabels = struct_dataset['test'][0,0]['images'], struct_dataset['test'][0,0]['labels']

TrainImages = TrainImages / 255.0
TestImages = TestImages / 255.0

TrainLabels = TrainLabels - 1
TestLabels = TestLabels - 1

TrainImages = TrainImages.reshape(len(TrainImages), 28, 28, 1)
TestImages = TestImages.reshape(len(TestImages), 28, 28, 1)

print("DataSet Loaded")
print(f"Train Images Shape: {TrainImages.shape}")
print(f"Train Labels Shape: {TrainLabels.shape}")
print(f"Test Images Shape: {TestImages.shape}")
print(f"Test Labels Shape: {TestLabels.shape}")

Loading EMNIST DataSet
DataSet Loaded
Train Images Shape: (124800, 28, 28, 1)
Train Labels Shape: (124800, 1)
Test Images Shape: (20800, 28, 28, 1)
Test Labels Shape: (20800, 1)


In [3]:
#Creating Model
print("Creating Model")

model = models.Sequential()

#Layer 1
model.add(layers.Conv2D(32 , (3,3),input_shape = (28,28,1),padding='same'))
model.add(layers.Activation("relu"))
model.add(layers.MaxPooling2D(pool_size = (2,2)))

#Layer 2
model.add(layers.Conv2D(32 , (3,3),padding='same'))
model.add(layers.Activation("relu"))
model.add(layers.MaxPooling2D(pool_size = (2,2)))

#Layer 3
model.add(layers.Conv2D(64 , (3,3),padding='same'))

#Layer 4
model.add(layers.Conv2D(64 , (3,3),padding='same'))

#Layer 5
model.add(layers.Flatten())

#Layer 6
model.add(layers.Dense(64))
model.add(layers.Activation("relu"))

#Dropout
model.add(layers.Dropout(0.2))

#Layer 7
model.add(layers.Dense(26))
model.add(layers.Activation("softmax"))

model.compile(loss = "sparse_categorical_crossentropy" , optimizer = "adam" , metrics = ["accuracy"])

print("Model Created")
print("Model Summary: ")
model.summary()

Creating Model
Model Created
Model Summary: 


/home/nkminion/miniconda3/envs/testvenv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 7, 7, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │         1,690 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 26)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 267,450 (1.02 MB)

 Trainable params: 267,450 (1.02 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
#Training and Saving model

print("Training Model")

EarlyStop = EarlyStopping(
	monitor = 'val_accuracy',
	patience = 5,
	verbose = 1,
	restore_best_weights = 1
)

Training = model.fit(TrainImages,TrainLabels,
					 batch_size = 128,
					 epochs = 100,
					 validation_data = (TestImages,TestLabels),
					 callbacks = [EarlyStop])
print("Training Complete")

model.save("EMNISTModel.keras")
print("Model Saved")

Training Model
Epoch 1/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.8036 - loss: 0.6340 - val_accuracy: 0.9075 - val_loss: 0.2837
Epoch 2/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8900 - loss: 0.3407 - val_accuracy: 0.9186 - val_loss: 0.2508
Epoch 3/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9053 - loss: 0.2892 - val_accuracy: 0.9256 - val_loss: 0.2265
Epoch 4/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9130 - loss: 0.2640 - val_accuracy: 0.9270 - val_loss: 0.2228
Epoch 5/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9177 - loss: 0.2492 - val_accuracy: 0.9325 - val_loss: 0.2039
Epoch 6/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9218 - loss: 0.2360 - val_accuracy: 0.9262 - val_loss: 0.2189
Epoch 7/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9241 - loss: 0.2258 - val_accuracy: 0.9321 - val_loss: 0.2068
Epoch 8/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.927

In [5]:
#Testing Model

print("Testing Model with inbuilt dataset")
TestLoss,TestAcc = model.evaluate(TestImages,TestLabels)
print(TestImages.shape)
print(TestImages.dtype)
print(np.min(TestImages), np.max(TestImages))
print(f"Mean: {np.mean(TestImages)}")
print(f"Test Accuracy: {TestAcc}")
print(f"Test Loss: {TestLoss}")

Testing Model with inbuilt dataset
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9392 - loss: 0.1965
(20800, 28, 28, 1)
float64
0.0 1.0
Mean: 0.1721735874638317
Test Accuracy: 0.939230740070343
Test Loss: 0.1965341866016388
